In [1]:
import numpy as np
import sys 

from autoencoder_v3 import loading_coef,encoder_forward,encoder_derivative
from sympy import symbols, simplify, derive_by_array
from scipy.integrate import solve_ivp
from sympy.physics.mechanics import *
from sympy import *
import sympy
import torch
import sys
import HLsearch as HL
import example_pendulum
import time


8.865152486035113e-66


In [2]:
device = 'cuda:0'
data = example_pendulum.get_pendulum_data(40)

In [3]:
num_sample = 100
training = True
save = False
noiselevel = 2e-2
file_route = R'C:\Users\87106\OneDrive\sindy\progress\AE'
load_date = '3-20'
load_ver = 1
widths = [1024,512,128]
params={}
params['ver'] = 1
params['date'] = '3-22'
params['widths'] = widths
params['activation'] = 'sigmoid'
params = loading_coef(params,load_date,load_ver,file_route,device)
image = torch.from_numpy(data['x']).to(torch.float32).to(device)
image_t = torch.from_numpy(data['dx']).to(torch.float32).to(device)
image_tt = torch.from_numpy(data['ddx']).to(torch.float32).to(device)
x = encoder_forward(image,params)
x = x.cpu().detach().numpy()
dx,ddx = encoder_derivative(image,image_t,image_tt,params)
dx = dx.cpu().detach().numpy()
ddx = ddx.cpu().detach().numpy()
X = []
Xdot = []

loading params
torch.Size([2601, 1024])
torch.Size([1024, 512])
torch.Size([512, 128])
torch.Size([128, 1])
torch.Size([1024])
torch.Size([512])
torch.Size([128])
torch.Size([1])
torch.Size([1, 128])
torch.Size([128, 512])
torch.Size([512, 1024])
torch.Size([1024, 2601])
torch.Size([128])
torch.Size([512])
torch.Size([1024])
torch.Size([2601])


In [ ]:
#adding noise
#not modified yet
"""
mu, sigma = 0, noiselevel
noise = np.random.normal(mu, sigma, X.shape[0])
for i in range(X.shape[1]):
    X[:,i] = X[:,i]+noise
    Xdot[:,i] = Xdot[:,i]+noise
"""

In [7]:
states_dim = 2
states = ()
states_dot = ()
for i in range(states_dim):
    if(i<states_dim//2):
        states = states + (symbols('x{}'.format(i)),)
        states_dot = states_dot + (symbols('x{}_t'.format(i)),)
    else:
        states = states + (symbols('x{}_t'.format(i-states_dim//2)),)
        states_dot = states_dot + (symbols('x{}_tt'.format(i-states_dim//2)),)
print('states are:',states)
print('states derivatives are: ', states_dot)
#Turn from sympy to str
states_sym = states
states_dot_sym = states_dot
states = list(str(descr) for descr in states)
states_dot = list(str(descr) for descr in states_dot)

states are: (x0, x0_t)
states derivatives are:  (x0_t, x0_tt)


In [11]:
#build function expression for the library in str
expr= HL.buildFunctionExpressions(2,states_dim,states,use_sine=True)
#expr=['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']
#expr=['cos(x0)','x0_t**2']
"a list of candidate function"
print(expr)
coef = torch.ones(len(expr), device=device).data.uniform_(-10,10)

['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']


In [9]:
#TBD
#time derivative of the expressions

In [ ]:
#TBD
#loss function